In [0]:
!pip install BeautifulSoup4 lxml html5lib

In [0]:
dbutils.library.restartPython()

In [0]:
import requests
import pandas as pd
import re
import io
from bs4 import BeautifulSoup
import time
import random
from pyspark.sql.functions import regexp_replace
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

In [0]:
TABLE_NAME = "finance_catalog.db_landing.src_raw_insiders_trx"

schema = StructType([
    StructField("filing", StringType(), True),
    StructField("symbol", StringType(), True),
    StructField("security", StringType(), True),
    StructField("reporting_name", StringType(), True),
    StructField("relationship", StringType(), True),
    StructField("trans_date", StringType(), True),
    StructField("purchase_sale", StringType(), True),
    StructField("shares", DoubleType(), True),
    StructField("price", DoubleType(), True),
    StructField("amount", DoubleType(), True),
    StructField("d_i", StringType(), True)
])

In [0]:
HEADERS_LIST = [
    {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1"
    },
    {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4; rv:109.0) Gecko/20100101 Firefox/115.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "DNT": "1"
    },
    {
        "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Mobile/15E148 Safari/604.1",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive"
    },
    {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/115.0.0.0 Safari/537.36",
        "Accept": "*/*",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Referer": "https://www.google.com/"
    }
]

In [0]:
headers = random.choice(HEADERS_LIST)

session = requests.Session()
session.headers.update(headers)

results = []
data_frames = []

base_url = "https://www.dataroma.com/m/ins/ins.php?t=d&am=0&sym=&o=fd&d=d"
response = session.get(base_url, timeout=10)
soup = BeautifulSoup(response.text, 'html.parser')

pages_div = soup.find("div", id="pages")

if pages_div is None:
    response = session.get(base_url, timeout=10)
    tables = pd.read_html(io.StringIO(response.text)) 
    trx_table = tables[2]

    if not trx_table.empty:
        results.append(trx_table)
else:
    links = pages_div.find_all('a')
    pages = [a.get_text() for a in links if a.get_text().strip().isdigit()]

    for page in pages:
        url = f"{base_url}&L={page}"
        response = session.get(url, timeout=10)
        time.sleep(random.uniform(1.5, 3.5))

        tables = pd.read_html(io.StringIO(response.text)) 
        trx_table = tables[2]
        results.append(trx_table)

if results:
    df = pd.concat(results, ignore_index=True)
    df.columns = [re.sub(r"\W+", "_", col).strip("_").lower() for col in df.columns]

    if data_frames:
        data_frames.append(df)
        df = pd.concat(data_frames, ignore_index=True).dropna(subset=['symbol'])

(
    spark
        .createDataFrame(df, schema=schema)
        .write
        .format("delta")
        .mode("append")
        .saveAsTable(TABLE_NAME)
)